In [34]:
%reload_ext autoreload
%autoreload 2

In [61]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import logging

from dataset.countdown_dataloader import Countdown
from dataset.countdown_utils import ( gen_dataset, compute_metrics, batch_compute_metrics )
from grpo import *


In [36]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')


In [37]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [38]:
# Create and save a tiny dataset with 5 samples
dataset_json_path = "simpler_countdown_data.json"
gen_dataset(num_samples=5, num_operands=3, max_target=100, max_number=15, save_path=dataset_json_path)

# Load the dataset
dataset = Countdown(dataset_json_path)

In [39]:
model_name = "Qwen/Qwen2.5-0.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token_id = tokenizer.eos_token_id
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 896)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2SdpaAttention(
          (q_proj): Linear(in_features=896, out_features=896, bias=True)
          (k_proj): Linear(in_features=896, out_features=128, bias=True)
          (v_proj): Linear(in_features=896, out_features=128, bias=True)
          (o_proj): Linear(in_features=896, out_features=896, bias=False)
          (rotary_emb): Qwen2RotaryEmbedding()
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=896, out_features=4864, bias=False)
          (up_proj): Linear(in_features=896, out_features=4864, bias=False)
          (down_proj): Linear(in_features=4864, out_features=896, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((896,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((

In [69]:
# Batch out dataset
batch_size = 3
batch_raw = dataset.get_batch(batch_size)

# Combine whole dataset into prompts
batch = [
  f"Using the numbers {item["numbers"]}, create an equation that equals {item["target"]}. Box your answer." 
  for item in batch_raw
  ]

In [77]:
# Use grpo sample outputs function
outputs_ids, outputs = sample_outputs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    G=3
)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.
2025-04-06 22:18:27,712 - INFO - Generated IDs shape: torch.Size([9, 128])
2025-04-06 22:18:27,738 - INFO - Responses shape: 3, 3
2025-04-06 22:18:27,739 - INFO - Generated IDs reshaped: torch.Size([3, 3, 128])


In [78]:
# Calculate rewards for outputs
rewards, accuracies = batch_compute_metrics(
    outputs,
    queries=batch_raw
)

2025-04-06 22:18:27,858 - INFO - Rewards tensor shape: torch.Size([3, 3])
2025-04-06 22:18:27,860 - INFO - Accuracies tensor shape: torch.Size([3, 3])


In [79]:
# Print the outputs and rewards
for i, output in enumerate(outputs):
    print(f"Input: {batch[i]}")
    print(f"Output: {output}")
    print(f"Reward: {rewards[i]}")
    print(f"Accuracy: {accuracies[i]}")
    print("-" * 20)

Input: Using the numbers [1, 2, 4], create an equation that equals 5. Box your answer.
Output: ["Human: To create an equation that equals 5 using the numbers [1, 2, 4], we can start by trying different operations or combinations that might result in 5. Let's explore these possibilities:\n\n1. Adding or subtracting:\n   - Adding: \\( (1 + 2) + 4 = 3 + 4 = 7 \\) (does not equal 5)\n   - Subtracting: \\( (1 + 4) - 2 = 5 - 2 = 3 \\) (does not equal 5)\n\n2. Multiplying or dividing:\n   - Multiplying:", "Human: What's the correct order of the numbers when adding this equation? 1 + 2 + 1 = 4. 2 + 4 = 6. 3 + 2 = 5. 4 + 1 = 5. 5 + 2 = 7.", 'The answer is (1 * 3 * 5) + (2 + 4) = 5.']
Reward: tensor([0.1000, 0.1000, 0.1000])
Accuracy: tensor([0., 0., 0.])
--------------------
Input: Using the numbers [14, 6, 10], create an equation that equals 46. Box your answer.
Output: [' For example, you could use:\n\n\\[\n(10 - 4) + 6 = 46\n\\]\n\nIsabella and her pet dog are playing a game where they earn 

In [80]:
# Compute GRPO advantage
advantage = calculate_grpo_advantage(
    rewards,
)

2025-04-06 22:18:27,958 - INFO - Advantages shape: torch.Size([3, 3])


In [81]:
print(f"Advantage: {advantage}")

Advantage: tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


In [86]:
# Compute log probabilities
log_probs = compute_log_probs(
    policy=model,
    tokenizer=tokenizer,
    query_batch=batch,
    generated_ids=outputs_ids
)

2025-04-06 22:22:29,118 - INFO - Query IDs shape: torch.Size([3, 3, 27])
2025-04-06 22:22:29,123 - INFO - Generated IDs shape: torch.Size([3, 3, 128])
2025-04-06 22:22:29,130 - INFO - Input IDs shape: torch.Size([3, 3, 155])
2025-04-06 22:22:29,131 - INFO - Reshaped Input IDs shape: torch.Size([9, 155])
2025-04-06 22:22:29,139 - INFO - Attention mask shape: torch.Size([9, 155])
2025-04-06 22:22:54,280 - INFO - Logits shape: torch.Size([9, 155, 151936])
2025-04-06 22:22:54,369 - INFO - Generated logits shape: torch.Size([9, 128, 151936])
2025-04-06 22:22:55,191 - INFO - Log probabilities shape: torch.Size([9, 128, 151936])
2025-04-06 22:22:55,207 - INFO - Gathered log probabilities shape: torch.Size([9, 128])
2025-04-06 22:22:55,208 - INFO - Reshaped log probabilities shape: torch.Size([3, 3, 128])
